#### Initialize Libraries

In [ ]:
from os import listdir
from os.path import isfile, join
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import itertools

!pip install fasttext
import fasttext

data_path = ''
import nest_asyncio
from twitchio.ext import commands
from twitchio.abcs import Messageable

data_path = 'Data/'
results_path = 'Results/'
dataset_path = data_path+'Datasets/'
train_path = data_path+'Train/'
test_path = data_path+'Test/'


#### Auxiliar functions

In [ ]:
def repeated_chars(word):
    clean_word=""
    for _, dups in (itertools.groupby(word)):
        seq = list(dups)
        clean_word += seq[0]
        if len(seq)>1:
            clean_word += seq[1]
    return clean_word

def cleanComment(comment):
    stemming = PorterStemmer()
    stops = set(stopwords.words("spanish"))
    ## START CODE
    line= comment.lower() ## Transform in lowercase
    line=re.sub('[:\[\]&%$\"\'!./,;:?=¿^\-#_*+)<>(¡@]','',line)
    line= line.split() ## Tokenize the text to get a list of terms
    line= [repeated_chars(word) for word in line]
    #line= [word for word in line if word not in stops]  ##eliminate the stopwords (HINT: use List Comprehension)
    line= [stemming.stem(word) for word in line] ## perform stemming (HINT: use List Comprehension)
    ## END CODE
    cleanedComment = ""
    for i, word in enumerate(line):
      if i!=0:
        cleanedComment += " "
      cleanedComment += word
    return cleanedComment

no pero tien culazo


## FASTTEXT 

#### Read Categorized Datasets

In [ ]:
fp = open(train_path+"Chat_Training_Model1.txt", "r", encoding="utf-8")    
other_comments = list()

line = fp.readline()
while(len(line) != 0):
    if "__label__0" in line:
        other_comments.append(line[len("__label__0"):])
    line = fp.readline()
fp.close()


fp = open(train_path+"Chat_Training_Model2.txt", "r", encoding="utf-8")
inappropriate_comments = list()
sexist_comments = list()

line = fp.readline()
while(len(line) != 0):
    if "__label__0" in line:
        inappropriate_comments.append(line[len("__label__0"):])
    elif "__label__1" in line:
        sexist_comments.append(line[len("__label__1"):])
    line = fp.readline()
fp.close()

##### Sexist Comments

In [ ]:
print("Number of sexist comments: ",len(sexist_comments))
print("Corresponds to a",len(sexist_comments)/(len(sexist_comments)+len(inappropriate_comments)+len(other_comments)),"% of the comments in the file.\n")
for comment in sexist_comments:
  print(comment)

##### Inappropiate Comments


In [ ]:
print("Number of inappropiate comments: ",len(inappropriate_comments))
print("Corresponds to a",len(inappropriate_comments)/(len(sexist_comments)+len(inappropriate_comments)+len(other_comments)),"% of the comments in the file.\n")
for comment in inappropriate_comments:
  print(comment)

#### FastText model

##### Training

In [ ]:
hyper_params = {"lr": 0.01,
    "epoch": 200,
    "wordNgrams": 2}  

model = fasttext.train_supervised(input=train_path+"Chat_Training_Model1.txt", **hyper_params)
model.save_model(results_path+"model1.bin")

model = fasttext.train_supervised(input=train_path+"Chat_Training_Model2.txt", **hyper_params)
model.save_model(results_path+"model2.bin")

##### Testing

In [ ]:
model = fasttext.load_model(results_path+"model2.bin")
model.test(test_path+"Chat_Testing_Model2.txt")

#### Other

In [ ]:
model = fasttext.load_model(results_path+"model1.bin")
model.predict(cleanComment("te gusta el sado o mas de chill"))

## TWITCH BOT

In [ ]:
text_file = open(dataset_path+"paraules_prohibides.txt", "r",encoding='utf-8')
paraules_prohibides= list()
comentaris=list()
frase=text_file.readline()

while(len(frase) != 0):
    frase2=frase.split("\n")
    paraules_prohibides.append(frase2[0])
    frase = text_file.readline()
text_file.close()

#inicialitzacio de les llistes necessaries pel bot
primer_warning=list()
segon_warning=list()
primer_baboso=list()
segon_baboso=list()
ban=list()
model = fasttext.load_model("results\model1.bin")
model2 = fasttext.load_model("results\model2.bin")

### Bot Class

In [ ]:
class Bot(commands.Bot):
    def __init__(self):
        nick = ""
        chanel = ""
        with open("config.secret","r") as fp:
            lines = fp.readlines()
            config = {}
            for line in lines:
                partes = line.replace("\n","").split("=")
                config[partes[0]] = partes[1]
            super().__init__(
                irc_token=config['TOKEN'],
                client_id =config['CLIENT_ID'],
                nick=config['BOT_NICK'],
                prefix=config['BOT_PREFIX'],
                initial_channels=[config['CHANNEL']]
            )
            self.nick = config['BOT_NICK']
            self.channel = config['CHANNEL']
            
    async def event_ready(self):
        print(f'Listo! | {self.nick}')
        ws = self._ws 
        await ws.send_privmsg(self.channel, f"/me has landed!")
    
  
    async def event_message(self, message):
        ban=0
        print(message.content)
        print(message.author.name)
        mensage_final_clean= cleanComment(message.content)
        #primer filtre
        mensage_final=mensage_final_clean.split()
        for mes in mensage_final:
            if(mes in paraules_prohibides ):
                ban=1
                ws = self._ws

                channel= self.channel

                #await ws.send_privmsg(channel, f".slow")
                if(message.author.name in primer_warning):
                    segon_warning.append(message.author.name)
                    primer_warning.remove(message.author.name)
                    

                    await ws.send_privmsg(channel, f".timeout "+ message.author.name +" 120")
                    
                elif (message.author.name in segon_warning):
                    segon_warning.remove(message.author.name)
                    ban.append(message.author.name)
                    await ws.send_privmsg(channel, f".ban "+ message.author.name )
                else:
                    primer_warning.append(message.author.name)
                    await ws.send_privmsg(channel, f".timeout "+ message.author.name +" 60")
                #await ws.send_privmsg(channel, f".timeout "+ message.author.name +" 60")

                comentaris_prohibits.append(message.content)
                
                break
        #segon filtre 
        print("llega")
        if(ban==0):
            ws = self._ws
            channel= self.channel
            print(mensage_final_clean)
            print(model.predict(mensage_final_clean))
            if(model.predict(mensage_final_clean)[0][0] =='__label__1' ):
                print(model2.predict(mensage_final_clean))
                if(model2.predict(mensage_final_clean)[0][0] =='__label__1' ):
                    if(message.author.name in primer_warning):
                        segon_warning.append(message.author.name)
                        primer_warning.remove(message.author.name)
                        await ws.send_privmsg(channel, f".timeout "+ message.author.name +" 120")
                    elif (message.author.name in segon_warning):
                        segon_warning.remove(message.author.name)
                        ban.append(message.author.name)
                        await ws.send_privmsg(channel, f".ban "+ message.author.name )
                    else:
                        primer_warning.append(message.author.name)
                        await ws.send_privmsg(channel, f".timeout "+ message.author.name +" 60")
                else:
                    
                    if(message.author.name in primer_baboso):
                        segon_baboso.append(message.author.name)
                        primer_baboso.remove(message.author.name)
                        await ws.send_privmsg(self.channel, f" " +message.author.name +" be careful my friend! ")

                    elif (message.author.name in segon_baboso):
                        segon_baboso.remove(message.author.name)
                        await ws.send_privmsg(channel, f".timeout "+ message.author.name +" 60")
                    else:
                        primer_baboso.append(message.author.name)
                        await ws.send_privmsg(self.channel, f" " +message.author.name +" be careful my friend! ")
        
    # Decorador para los comandos
    @commands.command(name='saludo')
    async def saludo(self, ctx):
        await ctx.send(f'Hola {ctx.author.name}!')

In [ ]:
nest_asyncio.apply()
bot = Bot()

bot.run()